<a href="https://colab.research.google.com/github/GirishHiremath/DL/blob/main/TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
ROOT ='/content/drive/MyDrive/Transfer_learning/transfer_learning'
os.chdir(ROOT)
assert os.getcwd() == ROOT

In [ ]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("fivethirtyeight")
%load_ext tensorboard

In [ ]:
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
tf.random.set_seed(42)
np.random.seed(42)

LAYERS = [ tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(300, kernel_initializer="he_normal"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dense(100, kernel_initializer="he_normal"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dense(10, activation="softmax")]


model = tf.keras.models.Sequential(LAYERS)

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-trai

In [ ]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid), verbose=2)

Epoch 1/10
1719/1719 - 7s - loss: 1.5275 - accuracy: 0.5970 - val_loss: 0.9444 - val_accuracy: 0.7980 - 7s/epoch - 4ms/step
Epoch 2/10
1719/1719 - 4s - loss: 0.7465 - accuracy: 0.8287 - val_loss: 0.5868 - val_accuracy: 0.8596 - 4s/epoch - 2ms/step
Epoch 3/10
1719/1719 - 4s - loss: 0.5412 - accuracy: 0.8624 - val_loss: 0.4685 - val_accuracy: 0.8834 - 4s/epoch - 2ms/step
Epoch 4/10
1719/1719 - 4s - loss: 0.4591 - accuracy: 0.8771 - val_loss: 0.4104 - val_accuracy: 0.8940 - 4s/epoch - 2ms/step
Epoch 5/10
1719/1719 - 4s - loss: 0.4142 - accuracy: 0.8869 - val_loss: 0.3758 - val_accuracy: 0.9006 - 4s/epoch - 2ms/step
Epoch 6/10
1719/1719 - 4s - loss: 0.3852 - accuracy: 0.8938 - val_loss: 0.3525 - val_accuracy: 0.9052 - 4s/epoch - 2ms/step
Epoch 7/10
1719/1719 - 4s - loss: 0.3644 - accuracy: 0.8980 - val_loss: 0.3348 - val_accuracy: 0.9102 - 4s/epoch - 2ms/step
Epoch 8/10
1719/1719 - 4s - loss: 0.3485 - accuracy: 0.9022 - val_loss: 0.3209 - val_accuracy: 0.9138 - 4s/epoch - 2ms/step
Epoch 9/

In [ ]:
model.save("pretrained_mnist_model.h5")

##Transfer Learning

In [ ]:
pretrained_mnist_model = tf.keras.models.load_model("pretrained_mnist_model.h5")

In [ ]:
pretrained_mnist_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-trai

In [ ]:
for layer in pretrained_mnist_model.layers:
  print(f"{layer.name}:{layer.trainable}")

flatten:True
dense:True
leaky_re_lu:True
dense_1:True
leaky_re_lu_1:True
dense_2:True


In [ ]:
for layer in pretrained_mnist_model.layers[:-1]:
  print(f"{layer.name}:{layer.trainable}")

flatten:True
dense:True
leaky_re_lu:True
dense_1:True
leaky_re_lu_1:True


In [ ]:
for layer in pretrained_mnist_model.layers[:-1]:
  layer.trainable = False
  print(f"{layer.name}:{layer.trainable}")

flatten:False
dense:False
leaky_re_lu:False
dense_1:False
leaky_re_lu_1:False


In [ ]:
for layer in pretrained_mnist_model.layers:
  print(f"{layer.name}:{layer.trainable}")

flatten:False
dense:False
leaky_re_lu:False
dense_1:False
leaky_re_lu_1:False
dense_2:True


##only last layer is trainable because we need to train only last layer

In [ ]:
lower_pretrained_layers = pretrained_mnist_model.layers[:-1]
new_model = tf.keras.models.Sequential(lower_pretrained_layers)

new_model.add(
    tf.keras.layers.Dense(2, activation="softmax")
)

In [ ]:
new_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_3 (Dense)             (None, 2)                 202       
                                                                 
Total params: 265,802
Trainable params: 202
Non-traina

In [ ]:
np.unique(y_train)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [ ]:
np.where(y_train%2 == 0, 1,0) 

array([0, 0, 1, ..., 0, 1, 1])

In [ ]:
print(y_train,y_test)

[7 3 4 ... 5 6 8] [7 2 1 ... 4 5 6]


In [ ]:
def update_even_odd_labels(labels):
  for idx, label in enumerate(labels):
    labels[idx] = np.where(label % 2 == 0, 1, 0)
  return labels

In [ ]:
y_train_bin, y_test_bin, y_valid_bin = update_even_odd_labels([y_train, y_test, y_valid])

In [ ]:
np.unique(y_valid_bin)

array([0, 1])

In [ ]:
new_model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

In [ ]:
history = new_model.fit(X_train, y_train_bin, epochs=10,
                    validation_data=(X_valid, y_valid_bin), verbose=2)

Epoch 1/10
1719/1719 - 4s - loss: 0.3898 - accuracy: 0.8288 - val_loss: 0.3247 - val_accuracy: 0.8676 - 4s/epoch - 2ms/step
Epoch 2/10
1719/1719 - 4s - loss: 0.3300 - accuracy: 0.8602 - val_loss: 0.3049 - val_accuracy: 0.8752 - 4s/epoch - 2ms/step
Epoch 3/10
1719/1719 - 4s - loss: 0.3163 - accuracy: 0.8660 - val_loss: 0.2948 - val_accuracy: 0.8796 - 4s/epoch - 2ms/step
Epoch 4/10
1719/1719 - 4s - loss: 0.3083 - accuracy: 0.8701 - val_loss: 0.2884 - val_accuracy: 0.8832 - 4s/epoch - 2ms/step
Epoch 5/10
1719/1719 - 4s - loss: 0.3023 - accuracy: 0.8725 - val_loss: 0.2834 - val_accuracy: 0.8846 - 4s/epoch - 2ms/step
Epoch 6/10
1719/1719 - 4s - loss: 0.2978 - accuracy: 0.8752 - val_loss: 0.2792 - val_accuracy: 0.8874 - 4s/epoch - 2ms/step
Epoch 7/10
1719/1719 - 4s - loss: 0.2939 - accuracy: 0.8772 - val_loss: 0.2758 - val_accuracy: 0.8872 - 4s/epoch - 2ms/step
Epoch 8/10
1719/1719 - 4s - loss: 0.2907 - accuracy: 0.8788 - val_loss: 0.2728 - val_accuracy: 0.8890 - 4s/epoch - 2ms/step
Epoch 9/

In [ ]:
new_model.evaluate(X_test, y_test_bin)

313/313 [==============================] - 1s 2ms/step - loss: 0.2768 - accuracy: 0.8861


[0.27683761715888977, 0.8860999941825867]

In [34]:
y_test[:3], y_test_bin[:3]

(array([7, 2, 1], dtype=uint8), array([0, 1, 0]))

In [36]:
X_new = X_test[:3]
np.argmax(new_model.predict(X_new), axis=-1)

1/1 [==============================] - 0s 68ms/step


array([0, 1, 0])